In [156]:
from PROPERTIES import *

from flask import Flask, request, make_response, Response, session
from flask_wtf.csrf import CSRFProtect, generate_csrf
from flask_sslify import SSLify
from uvicorn import Server, Config
from urllib.parse import quote
from ssl import SSLContext, PROTOCOL_TLS
import ssl
import subprocess
import logging
import random
import string
import uvicorn
import json
import uuid
import binascii
import hmac
import os
import OpenSSL
import datetime
import Cryptodome
from asgiref.wsgi import WsgiToAsgi
from cryptography.hazmat.primitives.asymmetric import ec, rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend, openssl
from cryptography.hazmat.primitives.kdf.concatkdf import ConcatKDFHMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography import x509
from cryptography.x509.oid import NameOID
from Cryptodome.Cipher import PKCS1_OAEP
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import RSA

In [157]:
# A Keys
ec_private_key_a = ec.generate_private_key(ec.SECP256R1(), default_backend())
ec_public_key_a = ec_private_key_a.public_key()

# Generate a new RSA private key
rsa_private_key_a = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Define the subject of the certificate
subject = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "DE"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, ""),
    x509.NameAttribute(NameOID.LOCALITY_NAME, ""),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, ""),
    x509.NameAttribute(NameOID.COMMON_NAME, "0.0.0.0"),
])

# Create the certificate
cert_a = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(subject)
    .public_key(rsa_private_key_a.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.utcnow())
    .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
    .add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    )
    .add_extension(
        x509.SubjectAlternativeName([x509.DNSName("127.0.0.1")]), critical=False,
    )
    .sign(rsa_private_key_a, hashes.SHA256(), default_backend())
)

# Print the private key and certificate in PEM format
private_key_pem_a = rsa_private_key_a.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption(),
)

cert_pem_a = cert_a.public_bytes(encoding=serialization.Encoding.PEM)

In [158]:
# B Keys
ec_private_key_b = ec.generate_private_key(ec.SECP256R1(), default_backend())
ec_public_key_b = ec_private_key_b.public_key()

# Generate a new RSA private key
rsa_private_key_b = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Define the subject of the certificate
subject = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "DE"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, ""),
    x509.NameAttribute(NameOID.LOCALITY_NAME, ""),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, ""),
    x509.NameAttribute(NameOID.COMMON_NAME, "0.0.0.0"),
])

# Create the certificate
cert_b = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(subject)
    .public_key(rsa_private_key_b.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.utcnow())
    .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
    .add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    )
    .add_extension(
        x509.SubjectAlternativeName([x509.DNSName("127.0.0.1")]), critical=False,
    )
    .sign(rsa_private_key_b, hashes.SHA256(), default_backend())
)

# Print the private key and certificate in PEM format
private_key_pem_b = rsa_private_key_b.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption(),
)

cert_pem_b = cert_b.public_bytes(encoding=serialization.Encoding.PEM)

In [159]:
################### STEP ONE ###################
# A gets ECDH Public Key from B and Computes Shared Secret
ec_public_key_b_hex = binascii.hexlify(ec_public_key_b.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)).decode('utf-8')
# B Computes Shared Secret
# Serializing Key:
serialized_ec_public_key_b = bytes.fromhex(ec_public_key_b_hex)
ec_public_key_peer_on_a = serialization.load_pem_public_key(
    serialized_ec_public_key_b,
    backend=default_backend()
)
# Exchanging Key:
ec_shared_key_on_a = ec_private_key_a.exchange(
    ec.ECDH(), ec_public_key_peer_on_a)
# Digesting Key:
ec_symmetric_key_on_a = ConcatKDFHMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    otherinfo=None,
    backend=default_backend()
).derive(ec_shared_key_on_a)

In [160]:
################### STEP TWO ###################
# B gets ECDH Public Key from A via Request
ec_public_key_a_hex = binascii.hexlify(ec_public_key_a.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)).decode('utf-8')
# B serializes Key
serialized_ec_public_key_a = bytes.fromhex(ec_public_key_a_hex)
ec_public_key_peer_on_b = serialization.load_pem_public_key(
    serialized_ec_public_key_a,
    backend=default_backend()
)
# Then computes shared secret and derives key via HKDF
ec_shared_key_on_b = ec_private_key_b.exchange(
    ec.ECDH(), ec_public_key_peer_on_b)
ec_symmetric_key_on_b = ConcatKDFHMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    otherinfo=None,
    backend=default_backend()
).derive(ec_shared_key_on_b)
# B compares digest messages:
# Convert the byte string to a hexadecimal string
cert_hex = binascii.hexlify(cert_pem_b).decode('utf-8')
# B Symmetrically Encrypts its' RSA Cert with Shared Secret
iv_on_b = os.urandom(16)
cipher = Cipher(algorithms.AES(ec_shared_key_on_b),
                modes.CBC(iv_on_b), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(bytes.fromhex(cert_hex))

In [161]:
################### STEP THREE ###################
# A Decryptes B's Public Rsa Key and Stores it
iv_on_a = iv_on_b
# A Symmetrically Decrypts Ciphertext with the Shared Secret as Key
decipher = Cipher(algorithms.AES(ec_shared_key_on_a),
                  modes.CBC(iv_on_a), backend=default_backend())
decryptor = decipher.decryptor()
rsa_public_key_b_on_a = decryptor.update(ciphertext)
# A Symmetrically and Asymmetrically Encrypts its' RSA Cert
# cert_bytes = binascii.unhexlify(rsa_public_key_b_on_a)
cert_b.public_bytes(encoding=serialization.Encoding.PEM)
rsa_public_key_b_on_a = (rsa_public_key_b_on_a.decode() + '---\n').encode()
cert_a = x509.load_pem_x509_certificate(cert_pem_a, default_backend())
cert_b_on_a = x509.load_pem_x509_certificate(rsa_public_key_b_on_a, default_backend())
cert_b_on_a_bytes = cert_b_on_a.public_bytes(encoding=serialization.Encoding.PEM)
padding_size = 2 * 32 + 2
nonce = b'\x00' * padding_size  # Generate a random nonce
associated_data=b'MIIDNDCCAhygAwIBAgIUGt0rXA1rc6gcdLMweq9Uvecjb9gwDQYJKoZIhvcNAQEL'
cert_pem_a = cert_a.public_bytes(encoding=serialization.Encoding.PEM)
chunk_size = (2048 // 8) - padding_size
chunks = [cert_b_on_a_bytes[i:i+chunk_size] for i in range(0, len(cert_b_on_a_bytes), chunk_size)]
ciphertext_sym_blocks = []
ciphertext_asym_blocks = []
# Asymmetric Encryption of Slized Blocks
cipher_rsa = PKCS1_OAEP.new(key=cert_b_on_a.public_key(), hashAlgo=hashes.SHA256())
for block in chunks:
    ciphertext_asym_blocks.append(cert_b_on_a.public_key().encrypt(block, padding=padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=associated_data
    )))
# Symmetric Encryption of Slized Blocks
cipher_aes = Cipher(algorithms.AES(ec_shared_key_on_a),
                modes.CBC(iv_on_a), backend=default_backend())
encryptor = cipher_aes.encryptor()
for asym_block in ciphertext_asym_blocks:
    ciphertext = encryptor.update(asym_block)
    ciphertext_sym_blocks.append(ciphertext)
sym_bytes = b''
# Byte Union
for sym_block in ciphertext_sym_blocks:
    sym_bytes = sym_bytes + sym_block

In [162]:
################### STEP FOUR ###################
# B Decryptes A's Public Rsa Key and Stores it
# B Splices Request Ciphertext into Sized Blocks
padding_size = (2 * 32) - 2
#chunk_size = (2048 // 8) - (2 * 32) - 2
chunk_size = 256
encrypted_sym_blocks = [sym_bytes[i:i+chunk_size] for i in range(0, len(sym_bytes), chunk_size)]
encrypted_asym_blocks = []
plaintext_blocks = []
plaintextDec = b''
# Symmetrical Decryption
decipher_aes = Cipher(algorithms.AES(ec_shared_key_on_b),
                modes.CBC(iv_on_b), backend=default_backend())
decryptor = decipher_aes.decryptor()
for asym_block in encrypted_sym_blocks:
    ciphertext_asym = decryptor.update(asym_block)
    encrypted_asym_blocks.append(ciphertext_asym)
# Asymmetrical Decryption
decipher_rsa = PKCS1_OAEP.new(key=rsa_private_key_b)
for sym_block in encrypted_asym_blocks:
    plaintext_blocks.append(rsa_private_key_b.decrypt(sym_block, padding=padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=associated_data
    )))
# Reassembly
for plaintext_block in plaintext_blocks:
    plaintextDec = plaintextDec + plaintext_block

In [163]:
print('send and received fully encrypted messages:')
requestEnc = ciphertext_sym_blocks
requestDec = encrypted_sym_blocks
print(str(requestEnc))
print(str(requestDec))

print('rsa encrypted data:')
rsaEncryptedEnc = ciphertext_asym_blocks
rsaEncryptedDec = encrypted_asym_blocks
print(str(rsaEncryptedEnc))
print(str(rsaEncryptedDec))

print('plaintext message:')
plaintextEnc = cert_b_on_a_bytes
plaintextDec = plaintextDec
print(str(plaintextEnc))
print(str(plaintextDec))

send and received fully encrypted messages:
[b'\x91\xa9E(\x16\xcf[U\x8a!\x872\xa7\x0b \xf16^\x0b6m\x10^\xda[e3\xefN\xc4\xb4\xca\r\x95\x0e\x985BD\xeb\x9d\x16\xae\x88_\x98\xee$\xad\xe0\xf6\xcd\x01\xdb\xbd\xb4\x88\xd9\xa7\x1e<\x95g\xa3w\x98\xd6G\x89\xf5\xf3mC\x9f\xa9\xa5\xc2\xd4\x12\xd6\x0f^P\x06\xdc\x8b\x8c\xd3\xc0\xd2c\x89\xd3yE\xa2c\x1a/=\x7f\xea\xd6\xd6X\xb4*\xec\xdeO9m\x9e"\xc9F\xba]\x95g\xf9S\x94x\t\x1c\xf54\x04X\xc3fk\xc8a1\xcd.\xb5\t\x898\xfao\x87Mb\xb5\rg\x0eA\xd7?\xafp\x0cMR\xbdOO\x88\xcfr\x8d\x1f\xd8\xf9\xcd?\xc7\xb1\xbd\xb4\n\x9e\x95\xbf\xbfUB^,\x96\xaec\xf2w+\xe7|&[\x114\xc4\xf8\xa4\xc5\x1b\x17\xe2\x1aNj\xce\x99\xcd\x11U"\xdaz\xc8\xe1\xb5\x01\x18\x01O\'\x92D\xcdB\xed\xab\xe3\x08Zo"\xcb\x8c\xf6\xd0e\x91\xbc?\r\x96\xc0\x14L\x10m\xec\x04\xc0\xeeW\x0fK\xe4', b'\xca\x0f\xaa\r^\x84\xa9aE\xce;`\xb6*a\xf6\x04\xd6\x97\x97N\xcd\x96\x01\xbb\x97\xfc8\xa35y?\xc0\x06\x01\x0f\xf0=\xdd\xbc\x11\x10H;\xdfOZ)\xa4\x8c2I\x8d\xe9\xec\xee\x9fL\x8b?\xe6w\x0cYe\xd2\x86\x03\x1a&\xf4J\x04\x00\x8b\x13i\